**Introduction**

If someone is looking to open a cafe in New York, where should they open it? <br/> At least two factors need to be taken into consideration: the number of cafes in a neighborhood and the number of residents in the neighborhood. <br/> Ideally, the new cafe should open in a neighborhood which has few cafes but a large number of residents.

**Data**

With Foursquare location data, one can easily obtain the number of cafes in each New York neighborhood. One could also find data online on the number of residents in each neighborhood in New York. <br/>
One can then calculate the cafe/capita index for each neighborhood and decide to open a new cafe in the neighborhood with the lowest cafe/capita. 